In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144
import pandas as pd
import numpy as np
import re
import traceback
from multiprocessing import Pool
pool = Pool(processes=16)
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import operator
from operator import itemgetter
import spacy

In [2]:
import itertools
import networkx as nx

In [3]:
from static_grader import grader

In [4]:
import pickle as pk
def dump_file(filename,data):
    with open(filename,'wb') as file:
        pk.dump(data,file)
def load_file(filename):
    with open(filename,'rb') as file:
        return pk.load(file)
from static_grader import grader    

In [88]:
link_list=load_file('link_list')

In [89]:
html_dict=load_file('html_dict')

In [188]:
def get_captions(page):
    soup=BeautifulSoup(page,'lxml')
    caption_divs= soup.find_all('font', attrs={'face': 'Verdana, Arial, Helvetica, sans-serif'}) 
    caption_divs.extend(soup.findAll('div', attrs={'class':'photocaption'}))
    #caption_divs.extend(soup.findAll('td', attrs={'class': 'photocaption'}))
    caption_divs.extend(soup.find_all("td", {'valign': 'top', 'class': 'photocaption'}, text=True))
    caption_divs.extend(soup.find_all("td", {'valign': 'top', 'class': 'photocaption', 'style': 'background-color:#faf9ee'}, text=True))
      
    return [i.text for i in caption_divs]
    

In [189]:
page=requests.get('http://www.newyorksocialdiary.com/party-pictures/2015/celebrating-the-neighborhood').text
test_caption=get_captions(page)

In [190]:
print(len(test_caption))

109


In [171]:
#https://github.com/hazlanhamdan/social_graph/blob/master/graph_analyze.py

def extract_names(caption):
    '''
    Extract a list of names from a caption string.  
    Names should be in format 'Firstname Lastname'.  
    If no names extracted, return None.  
    This function splits up the string using 'when', 'and', and commas
    and passes the resulting segments to the function `parse_names`
    '''
    # Remove leading and trailing whitespace
    caption = re.sub('[;\.]', '', caption)
    match = re.search('^\s*([^\s].+[^\s])\s*$', caption, re.DOTALL)
    if match:
        caption = match.group(1)
    else: 
        print (caption)
        print ('NO CHARACTERS FOUND')
        return None
    
    # If too long, throw out the caption
    if len(caption) > 250:
        print ('TOO LONG')
        return None
        
    # initialize the list
    names = []
    
    # split on pattern '... with ...'
    # if there's a match, parse the segment before the 'with',
    # leave the rest to be split below
    segments = re.split('\swith\s', caption)
    if len(segments)==2:
        new_names = parse_names(segments.pop(0))
        if new_names:
            for nm in new_names:
                if nm != None: names.append(nm)
    elif len(segments)>2:
        print ('***** UNHANDLED CASE: MORE THAN 1 "WITH" *****')
        print (caption)
        return None
    # after this, segments have length 1
        
    # split on the pattern 'name, name, and name'
    segments = re.split('\s?,\s?(?:and)?\s?', segments[0])
        
    # if not split already, split on pattern 'name and name',
    # but ONLY IF it's not a 'jack and jane doe' case
    if len(segments)==1:
        test_segs = re.split('\sand\s', segments[0])
        # 'jack and jane doe' case indicated by no whitespace in 
        # the first test_seg
        if re.search('\s', test_segs[0]):
            segments = test_segs
    
    #parse all the segments
    for sg in segments:
        new_names = parse_names(sg)
        if new_names:
            for nm in new_names:
                if nm != None: names.append(nm)
    
    return names

def parse_names(seg):
    '''
    Parse the names in segments generated in `extract_names`.
    If parsing fails, return None.
    Except for a few special cases, each segment is assumed to contain
    only one name. 
    '''
    # remove text in parentheses
    seg = re.sub(r'^\([^\)]*\)\s', '', seg) # bol
    seg = re.sub(r'\s\([^\)]*\)\.?$', '', seg) # eol
    seg = re.sub(r'\s\([^\)]*\)\s', ' ', seg) # elsewhere
    seg= seg.replace(" von"," Von") # try  " von " and " van "
    seg= seg.replace(" van"," Van")
    seg= seg.replace(" de"," De")
    # All subsequent processing is word by word....
    words = seg.split()#[i.strip() for i in seg.split(",")]
    #print(words)
    
    titles = ('Dr.', 'Mayor', 'Mr.', 'Mrs.', 'Ms.', 'Sir', 'Justice', 'Governor',
              'Prince', 'Princess', 'King', 'Queen', 'Count', 'Countess', 'Baroness')
    
    flags = ('Honoree','Chair','Co-Chair','Trustee','Director','Chairman',\
             'Chairwoman','CEO','Designer','Administrator','Winner','Fellow',\
             'Congressman','Actress','Playwright','Curator','Laureate','Member',\
             'Governor','Representative','Host','Vice-Chair',
             'Rev.','Reverend','Conductor','Mayor','Co-Chairs','Lady',\
             'Designer/artist','President','Recipient','Editor',
             'Commissioner','General','Speaker','Teacher','Sgt.')
    
    if len(words) > 6 and check_caps(words[0]) and check_caps(words[1]):
        return [' '.join([words[0], words[1]])]
    if len(words)>2 and (words[-3] in flags or re.search(r':$', words[-3]) or re.search(r"'s", words[-3])):
        words = words[-2:]
    elif len(words)>3 and (words[-4] in flags or re.search(r':$', words[-4]) or re.search(r"'s", words[-4])):
        words = words[-3:]
    
    # Special case: '[Tt]he Honorable'
    if len(words)==4 and (words[0]=='the' or words[0]=='The') and words[1]=='Honorable':
        return [' '.join([words[2], words[3]])]
    
    # Special case: 'jack and jane doe'
    if len(words)==4 and words[1]=='and':
        if not all(check_caps(wd) for wd in [words[0],words[2],words[3]]):
            return None
        return [' '.join([words[0], words[3]]), ' '.join([words[2], words[3]])]
    
    # Special case: 'mr. and mrs. jane doe'
    if len(words)==5 and words[0] in titles and words[2] in titles and words[1]=='and':
        if not all(check_caps(wd) for wd in words[3:]):
            return None
        return [' '.join([words[3], words[4]])]
    
    # If not capitalized like a name, throw out
    if not all(check_caps(wd) for wd in words):
        return [None]
    
    if len(words)==1:
        return [None]
    elif len(words)==2:
        return [' '.join(words)]
    elif len(words)==3:
        if words[0] in titles:
            return [' '.join(words[1:])]
        else:
            if check_caps(words[1]):
                return [' '.join(words)]
            else:
                return [' '.join([words[0], words[2]])]
            
            
    elif len(words)==4 and words[0] in titles:
        return [' '.join([words[1], words[3]])]
    
    
    else:
        print ()
        print (words)
        print ('***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****')
        print ()
        #print seg
        return [None]

def check_caps(word):
    
    '''
    Return a boolean indicating whether a putative word in a name follows 
    the right capitalization conventions
    We require that the first alphanumeric character be uppercase (leading
    quotations and other punctuation are allowed) and that it is followed 
    by at least one lowercase character.  
    '''
    # get the first letter, False if none found
    match = re.search('\w', word)
    if not match: 
        return False
    first_let = match.group()
    # First letter must be uppercase
    if not first_let.isupper():
        return False
    # There must be at least one lowercase letter
    match = re.search('[a-z]', word)
    if not match: 
        return False
    return True

In [ ]:
from multiprocessing import Pool
pool = Pool(processes=16)

In [ ]:
Name1=pool.map(extract_names,test_caption)

In [191]:
Name1=list(map(extract_names,test_caption))


['The', 'Waldorf', 'Astoria', 'Ballroom']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****



In [192]:
Name2=list(set(sum(Name1,[])))

In [193]:
Name2.sort()
Name2[:10]

['Alberto Villalobos',
 'Alex Papachristidis',
 'Alexandra Richards',
 'Alexis Mersentes',
 'Allison Minton',
 'Amory McAndrew',
 'Andi Potamkin',
 'Angela Clofine',
 'Anne Strickland',
 'Barbara Regna']

In [170]:
sentence = "try to be nice; underastand people. start fresh."
sentence = re.sub('[;\.]', '', sentence)
sentence

'try to be nice underastand people start fresh'

In [194]:
def sample_names():
    return Name2[:100]

grader.score(question_name='graph__sample_names', func=sample_names)

Your score:  1.0


In [195]:
dump_file('Name2',Name2)

In [156]:
def get_all_captions(html_dict):
    return list(map(get_captions,list(html_dict.values())))

In [196]:
All_captions_110=get_all_captions(html_dict)

In [197]:
dump_file('All_captions_110',All_captions_110)

In [159]:
def extract_all_names(All_captions):
    All_names=[]
    for acaption in All_captions:
        All_names.append(list(map(extract_names,acaption)))
    return All_names
        

In [198]:
All_names_110=extract_all_names(All_captions_110)


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

***** UNHANDLED CASE: MORE THAN 1 "WITH" *****
Lara getting ready in the bridal suite with a little help from with her mother and sister

['More', 'Greater', 'New', 'York', 'Links']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Hon', 'Cynthia', 'Brown', 'Franklin']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Junoon', '"The', 'Pakistani', 'Rock', 'Star', 'Professor"']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dress', 'For', 'Success', 'Honorees']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Callie', 'Thorne', 'Callie', 'Thorne']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Frederique', 'Van', 'Der', 'Wal']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN



['Jerey', 'Boys', 'Christian', 'Hoff']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Anne', 'Teresa', 'De', 'Keersmaeker']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Little', 'Red', 'Riding', 'Hood']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Little', 'Red', 'Riding', 'Hood']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Begonia', 'Santos', 'El', 'Koury']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dr', 'Carlos', 'Cordon', 'Cardo']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Bogonia', 'Santos', 'El', 'Koury']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Begonia', 'Santos', 'El', 'Koury']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['The', 'Hiltz', 'Family:', 'William']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['New', 'York', 'City', 'Girl', 'Scouts']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['The', 'El', 'Museo', 'Group']
***** UNHANDLED CASE: SEGME


TOO LONG

NO CHARACTERS FOUND
 
NO CHARACTERS FOUND

NO CHARACTERS FOUND

['Juan', 'Carlos', 'Arcila', 'Duque']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Juan', 'Carlos', 'Arcila', 'Duque']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Juan', 'Carlos', 'Arcila', 'Duque']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Juan', 'Carlos', 'Arcila', 'Duque']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Juan', 'Carlos', 'Arcila', 'Duque']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dance', 'Legend', 'Marge', 'Champion']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Barbara', 'Fox', 'Lady', 'Bordiga']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

['James', 'Mishka', 'Suzie', 'John


['Westchester', 'County', 'Executive', 'Andy', 'Spano']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Westchester', 'County', 'Executive', 'Andy', 'Spano']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Jean', 'Georges', 'Alfred', 'Portale']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Patina', 'Restaurant', 'Group/', 'Patina', 'Restaurant']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


NO CHARACTERS FOUND

['Cold', 'SPring', 'Harbor', 'Association', 'Director']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

***** UNHANDLED CASE: MORE THAN 1 "WITH" *****
John Anton, Pedigree, Mary Tyler Moore with Nebraska, Geoff Brown, Pedigree, Bernadette Peters with Fluffy, Annemarie Lucas, ASPCA, and Ed Sayres, ASPCA, both with Dumpling

['Former', 'New', 'York', 'Giant', 'Howard', 'Cross']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Court', 'TV’s', 'Jack', 'Ford']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Former',

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['City', 'Councilman', 'Dan', 'Garodnick']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Harpist', 'José', 'Luis', 'Rodriguez']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['NY-Presbyterian/Weill', 'Cornell', 'Neurology', 'Team']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG
TOO LONG

['Rabiana', 'De', 'Lande', 'Long']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Anne', 'Marie', 'Van', 'Tricht']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Gala', 'Chairs', 'Jane', 'Pollock']
***** UNHANDLED CASE

...



['Mahsa', 'Pelosky', 'Danielle', 'Hootnick']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Fe', 'Fendi,', 'Debbie', 'Bancroft,', 'Paola', 'Fendi,']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Anne-Rose', 'Van', 'Den', 'Bossche']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dayssi', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Elizabeth', 'Costa', 'De', 'Rusch']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Gala', 'Co-Chairman', 'Jo', 'Sullivan', 'Loesser']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Gala', 'Co-Chairman', 'Jo', 'Sullivan', 'Loesser']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Gala', 'Co-Chairmen', 'Jo', 'Sullivan', 'Loesser']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Gregory', 'Delli', 'Carpini', 'Jr']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Chris', 'Lee', 'PatrickMcMullancom', 'Sunny', 'Norton']
***** UNHANDLED CASE

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['State', 'Assemblyman', 'Daniel', 'O’Donnell']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Event', 'Chairs', 'Jamie', 'Mitchell']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Mt', 'Carmel', 'Holy', 'Rosary', 'Violin']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Catherine', 'McCormick', 'Priscilla', 'Rattazzi']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Renowned', 'Flautist', 'Wendy', 'Luck']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Drama', 'Desk', 'Lifetime', 'Achievement', 'Award', 'Winner']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Joseph', "'Rev", "Run'", 'Simmons']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Joesph', '"Rev', 'Run"', 'Simmons']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dr', 'Myat', 'Myat', 'Mon']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dr', 'Myat', 'Myat', 'Mon']
***** UNHAND

***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Kasseem', '"Swizz', 'Beatz"', 'Dean']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Kasseem', '"Swizz', 'Beatz"', 'Dean']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Rye', 'City', 'Councilwoman', 'Catherine', 'Parker']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Rocky', 'Reef', 'Farm', 'Pumpkins']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND

['Daisy', 'Olarte', 'De', 'Kanavos']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Lisa', 'Firestone', 'Van', 'Winterfeldt']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Lisa', 'Firestone', 'Van', 'Winterfeldt']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Joanna', 'Baker', 'De', 'Neufville']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Joanna', 'Baker', 'De', 'Neufville']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

['First', 'Presbyterian', 'Church


['Peter', 'Ambrose', 'Events', 'Catering']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

***** UNHANDLED CASE: MORE THAN 1 "WITH" *****
Valerie Angeli with DaVinci, Candy Udell, Jill Rappaport with 
            with Rubie, and Jennifer Judkins

['Raul', 'Montes', 'De', 'Oca']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Hostess', 'Mary', 'Jane', 'Brock']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Maria', 'De', 'Jesus', 'Castellon']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Gabby', 'Karan', 'De', 'Felice']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND
***** UNHANDLED CASE: MORE THAN 1 "WITH" *****
Association president Claire Knopf with her husband Woody, with Stevie and Todd Thompson

['Jean', 'Doyen', 'De', 'Montaillou']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Jean', 'Doyen', 'De', 'Montaillou']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Jean', 'Doyen', 'De', 'Montaill


['Citymeals', 'Board', 'Vice', 'President']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Jean', 'Remmel', 'Little', 'FitzSimmons']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Jonelle', 'Procope', 'Brett', 'Wright']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

['New', 'York', 'City', 'Chapter']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['New', 'York', 'City', 'Chapter']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['New', 'York', 'City', 'Chapter']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

TOO LONG

['New', 'York', 'City', 'Chapter']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****

 
NO CHARACTERS FOUND

['Blair', 'Clarke', 'Melyora', 'Kramer']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Dr', 'Alessandro', 'Di', 'Rocco']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Patrick', 'Baugier', 'De', 'Chevestre']
***** UNHANDLED CASE: SEGMENT PATTERN UNMATCHED *****


['Patrick'

In [199]:
print(len(All_names_110),len(All_captions_110))

1193 1193


In [41]:
#dump_file('All_names_110',All_names_110)

In [200]:
print(len(sum(All_names_110,[])))

102758


In [42]:
print(len(sum(All_names_110,[])))

131075


In [201]:
people_names=sum(All_names_110,[])

In [202]:
len(people_names)

102758

In [289]:
def gen_paris(data):
    data_combo=[]
    for alist in data:
        try:
            if len(alist) > 1:
                data_combo.append(list(itertools.combinations(alist,2)))
        except Exception as e:
            print (str(e))
            pass
    return data_combo

In [203]:
data_combo = []
for line in people_names:
 
    try:
        if str(line) == str(['']):
            pass #print len(line) #pass
        else:
            if len(line) == 1:
                pass
            else:
                #print "yes", line
                k=(list(itertools.combinations(line,2)))    #creating all possible two tuple combinations 
                                                            #of names that are in a picture 
                data_combo.append(k)
                # added a statement not to append if one name only
                # think I did this twice then! data vs list_of_words
    
    except:   
        pass

In [296]:
(list(itertools.combinations(people_names[4],2)))

[('Sylvana Soto-Ward', 'Luisiana Mendoza')]

In [301]:
gen_paris(people_names[:20]) 

[[('Jessie Maxwell', 'Jason Eblances')],
 [('Amy McFarland', 'Arden Wohl')],
 [('Sylvana Soto-Ward', 'Luisiana Mendoza')],
 [('Frederic Fekkai', 'Shirin Von Wulffen')],
 [('Jennifer Creel', 'Mark Gilbertson'),
  ('Jennifer Creel', 'Ashley McDermott'),
  ('Mark Gilbertson', 'Ashley McDermott')],
 [('Leven Rambin', 'Darcy Jones'),
  ('Leven Rambin', 'Tracy Stern'),
  ('Darcy Jones', 'Tracy Stern')],
 [('Olivia Chantecaille', 'Adelina Wong Ettelson')],
 [('Olivia Palermo', 'Susan Shin')],
 [('Kenya Clarke', 'Monique Jones'),
  ('Kenya Clarke', 'Mary Brown'),
  ('Monique Jones', 'Mary Brown')],
 [('Gillian Miniter', 'Arthur Bakal')],
 [('Kim Hicks', 'Cherell Flaherty')],
 [('Amy Smilovic', 'Emilia Fanjul Pfeifler'),
  ('Amy Smilovic', 'Liz Walker'),
  ('Emilia Fanjul Pfeifler', 'Liz Walker')],
 [('Andrew Saffir', 'Nina Griscom')]]

In [299]:
data_combo[:10]

[[('Jessie Maxwell', 'Jason Eblances')],
 [('Amy McFarland', 'Arden Wohl')],
 [('Sylvana Soto-Ward', 'Luisiana Mendoza')],
 [('Frederic Fekkai', 'Shirin Von Wulffen')],
 [('Jennifer Creel', 'Mark Gilbertson'),
  ('Jennifer Creel', 'Ashley McDermott'),
  ('Mark Gilbertson', 'Ashley McDermott')],
 [('Leven Rambin', 'Darcy Jones'),
  ('Leven Rambin', 'Tracy Stern'),
  ('Darcy Jones', 'Tracy Stern')],
 [('Olivia Chantecaille', 'Adelina Wong Ettelson')],
 [('Olivia Palermo', 'Susan Shin')],
 [('Kenya Clarke', 'Monique Jones'),
  ('Kenya Clarke', 'Mary Brown'),
  ('Monique Jones', 'Mary Brown')],
 [('Gillian Miniter', 'Arthur Bakal')]]

In [236]:
# Create graph
import operator
eM = nx.MultiGraph()
Gee = nx.Graph()

Multi_social=nx.MultiGraph()
def build_graph(data,single=False):
    my_graph=nx.MultiGraph()
    
    if single==True:
        my_graph=nx.Graph()
    for i in data_combo:
        try:
            my_graph.add_edges_from(i,weight=1)
        except Exception as e:
                print (str(e))
                pass
    return my_graph

In [237]:
Multi_social=build_graph(data_combo)

In [238]:
Single_social=build_graph(data_combo,single=True)

In [242]:
Degree=list(Multi_social.degree)
Sorted_degree=sorted(Degree,key= lambda x:(-x[1],x[0]))
Sorted_degree[:10]
                     

[('Jean Shafiroff', 631),
 ('Mark Gilbertson', 500),
 ('Gillian Miniter', 498),
 ('Geoffrey Bradfield', 371),
 ('Alexandra Lebenthal', 343),
 ('Somers Farkas', 322),
 ('Andrew Saffir', 292),
 ('Debbie Bancroft', 290),
 ('Yaz Hernandez', 284),
 ('Eleanora Kennedy', 278)]

In [213]:
dump_file('Sorted_degree_110',Sorted_degree)

In [277]:
Sorted_degree[:20]

[('Jean Shafiroff', 631),
 ('Mark Gilbertson', 500),
 ('Gillian Miniter', 498),
 ('Geoffrey Bradfield', 371),
 ('Alexandra Lebenthal', 343),
 ('Somers Farkas', 322),
 ('Andrew Saffir', 292),
 ('Debbie Bancroft', 290),
 ('Yaz Hernandez', 284),
 ('Eleanora Kennedy', 278),
 ('Sharon Bush', 270),
 ('Kamie Lightburn', 261),
 ('Jamee Gregory', 255),
 ('Alina Cho', 246),
 ('Muffie Potter Aston', 246),
 ('Mario Buatta', 231),
 ('Allison Aston', 228),
 ('Bonnie Comley', 225),
 ('Lucia Hwong Gordon', 223),
 ('Michael Bloomberg', 222)]

In [243]:
def degree():
    return Sorted_degree[:100]

grader.score(question_name='graph__degree', func=degree)

Your score:  1.0


In [20]:
pgrank_scipy=nx.pagerank_scipy(Multi_social,alpha=.6)
pgsorted_scipy = sorted(pgrank_scipy.items(), key=operator.itemgetter(1), reverse=True)

In [21]:
pgsorted_scipy

[('Jean Shafiroff', 0.0004230944670694563),
 ('Mark Gilbertson', 0.0003430175327289061),
 ('Gillian Miniter', 0.00027359902915124773),
 ('Geoffrey Bradfield', 0.0002528865046904302),
 ('Andrew Saffir', 0.00023836069908707226),
 ('Alexandra Lebenthal', 0.0002126424236497223),
 ('Mario Buatta', 0.0002105533472281826),
 ('Sharon Bush', 0.00020045202814112737),
 ('Somers Farkas', 0.00019504706471525375),
 ('Debbie Bancroft', 0.00019016795355286849),
 ('Michael Bloomberg', 0.00018989357734400413),
 ('Yaz Hernandez', 0.0001885071972758065),
 ('Kamie Lightburn', 0.00018827295342025945),
 ('Lucia Hwong Gordon', 0.00017843738315733847),
 ('Alina Cho', 0.0001690250766627524),
 ('Lydia Fenet', 0.0001669383036331056),
 ('Eleanora Kennedy', 0.00016475837892319782),
 ('Liliana Cavendish', 0.00016081188874281859),
 ('Bonnie Comley', 0.00015529920384430194),
 ('Martha Stewart', 0.00015503145244109379),
 ('Barbara Tober', 0.00015376496728323398),
 ('Patrick McMullan', 0.00015053402057521752),
 ('Kipton

In [248]:

pgrank=[]
pgsorted=[]
pgrank=nx.pagerank(Single_social,alpha=0.65)
pgsorted = sorted(pgrank.items(), key=operator.itemgetter(1), reverse=True)
print (pgsorted[1:10])

[('Mark Gilbertson', 0.0003610670741573026), ('Andrew Saffir', 0.00023931487388173484), ('Gillian Miniter', 0.00023915630787053492), ('Geoffrey Bradfield', 0.00023456934853562287), ('Mario Buatta', 0.00023296786202491358), ('Alexandra Lebenthal', 0.00020703129380314928), ('Michael Bloomberg', 0.00020490204995674997), ('Sharon Bush', 0.00019888303845166136), ('Somers Farkas', 0.00019057067084566365)]


In [216]:
dump_file('pgsorted_110',pgsorted)

In [22]:
def pagerank():
    return pgsorted_scipy[:100]

grader.score(question_name='graph__pagerank', func=pagerank)

Your score:  1.0


In [48]:
from collections import Counter

In [274]:
Edges_lists=[(node1,node2) for node1,node2,_ in Multi_social.edges(data=True)]
Counter(Edges_lists).most_common(10)

In [56]:
Edges_lists1=[(node1,node2) for node1,node2,_ in Single_social.edges(data=True)]

In [58]:
Counter(Edges_lists1).most_common(10)

[(('Jessie Maxwell', 'Jason Eblances'), 1),
 (('Amy McFarland', 'Arden Wohl'), 1),
 (('Amy McFarland', 'David Stark'), 1),
 (('Amy McFarland', 'Marisa Brown'), 1),
 (('Amy McFarland', 'Ulrica Lanaro'), 1),
 (('Amy McFarland', 'Marisa Arredondo'), 1),
 (('Amy McFarland', 'Sean Kingston'), 1),
 (('Amy McFarland', 'Susan Magazine'), 1),
 (('Amy McFarland', 'Vanessa Von Bismark'), 1),
 (('Amy McFarland', 'Rory Hermelee'), 1)]

In [276]:
def best_friends():
    return Counter(Edges_lists).most_common(100)

grader.score(question_name='graph__best_friends', func=best_friends)

Your score:  0.9700000000000006


In [220]:
dump_file('edges_sorted_110',edges_sorted)

In [278]:
len(data_combo)

78077

In [284]:
Combined_names=sum(sum(All_names_110,[]),[])

TypeError: can only concatenate list (not "NoneType") to list

In [283]:
Combined_names[1]

['Annalise Peterson']

In [288]:
len(Multi_social.edges)

185946

In [302]:
def get_captions():
    base_url = 'http://www.newyorksocialdiary.com'
    page = requests.get('http://www.newyorksocialdiary.com/party-pictures/2007/great-chefs-and-great-film-0')
    soup = BeautifulSoup(page.text, 'lxml')
    captions = soup.find_all(attrs={'class':"photocaption"})
    return captions

In [303]:
a=get_captions()

In [5]:
Multi_social=nx.read_gpickle('Multi_social')

In [50]:
Single_social=nx.read_gpickle('Social_social')

In [305]:
nx.write_gpickle(Single_social,'Social_social')

In [306]:
nx.write_gpickle(Multi_social,'Multi_social')

In [54]:
#import igraph as ig
#print (ig.__version__)

In [52]:
#!pip install igraph

In [ ]:
nx.draw(Multi_social, with_labels=True)
plt.show()

/opt/conda/envs/data3/lib/python3.6/site-packages/networkx/drawing/layout.py:499: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt((delta**2).sum(axis=0))
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/conda/envs/data3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/data3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/data3/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/opt/conda/envs/data3/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/opt/conda/envs/data3/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/opt/conda/envs/data3/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/opt/conda/envs/data3/lib/pytho

In [4]:
#!pip install line_profiler

In [5]:
%load_ext line_profiler

In [45]:
def spaCy_namelist_2D(input_listOfCaptionStrings):
    #### load (once for whole process)
    nlp = spacy.load('en_core_web_lg',disable=['textcat','parser',\
                                               'tagger','arbitrary_fixup_rules','entity'\
                                              ]) # spaCy library, stripped of a useless component

    ### Loop over captions
    photo_list = list([]) # vertical list
    for i in range(0,len(input_listOfCaptionStrings)):
        
        #t1 = time.time()
        doc = nlp(input_listOfCaptionStrings[i]) # process one caption
        gen = (ent for ent in doc.ents if ent.label_=='PERSON') # gen function: people in that caption
        people_in_photo = []
        for ent in gen:
            people_in_photo.append(ent.text.strip()) #horizontal list: people in this photo caption
        photo_list.append(people_in_photo) # list of ppl in photo, by photo
        #print(f'{i:4} | {len(people_in_photo):3} people found in {(time.time()-t1):4.2f} sec')
    ### Do some full list edits that I don't need to mention ####
    return photo_list

In [42]:
All_captions_110=load_file('All_captions_110')

In [46]:
a=spaCy_namelist_2D(All_captions_110[0])

In [47]:
a[:10]

[['Tatiana Boncampagni'],
 ['Annalise Peterson'],
 ['Jessie Maxwell'],
 ['Amy McFarland', 'Arden Wohl'],
 ['Sylvana Soto-Ward', 'Luisiana Mendoza'],
 ['Frederic Fekkai', 'Shirin von Wulffen'],
 ['Jennifer Creel', 'Mark Gilbertson', 'Ashley McDermott'],
 ['Leven Rambin', 'Darcy Jones', 'Tracy Stern'],
 ['Olivia Chantecaille', 'Adelina Wong Ettelson'],
 ['Olivia Palermo', 'Susan Shin']]

In [21]:
a=spaCy_namelist_2D(All_captions_110[0])

In [26]:
10/91

0.10989010989010989

In [51]:
#%lprun -f spaCy_namelist_2D(All_captions_110[0])


In [31]:
a[:10]

[[], [], [], [], [], [], [], [], [], []]